In [7]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark.functions import *
#desc, abs, count, col, concat, concat_ws, udf, array_construct, countDistinct, when
from snowflake.snowpark.window import Window
from snowflake.snowpark.types import *

from sklearn.model_selection import train_test_split # split  data into training and testing sets
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer # for scoring during cross validation
from sklearn.model_selection import GridSearchCV # cross validation
from sklearn.metrics import confusion_matrix # creates a confusion matrix
#from sklearn.m etrics import plot_confusion_matrix # draws a confusion matrix
from sklearn.metrics import mean_squared_error as MSE

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

import xgboost as xgb 
#import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import math

In [12]:
today = datetime.today()
current_month_start = datetime(today.year, today.month, 1)
previous_month_start = datetime(today.year, today.month-1, 1)
#previous_month_end = current_month_start - timedelta(days=1)

In [15]:
previous_month_start.date

<function datetime.date>

In [14]:
query1 = """DELETE FROM TABLE DATA_LAB_TEST.PREDICTOR.TR_ORDERS_DEVICE WHERE date_column >= '{0}'"""
query1.format(previous_month_start)

"DELETE FROM TABLE DATA_LAB_TEST.PREDICTOR.TR_ORDERS_DEVICE WHERE date_column >= '2023-05-01 00:00:00'"

In [16]:
table = session.sql(query1.format(previous_month_start))
table.collect()

Failed to execute query [queryID: 01ad0254-0001-2715-0000-7181f9b37e5e] DELETE FROM TABLE DATA_LAB_TEST.PREDICTOR.TR_ORDERS_DEVICE WHERE date_column >= '<built-in method date of datetime.datetime object at 0x7fc5398da210>'
001003 (42000): SQL compilation error:
syntax error line 1 at position 12 unexpected 'TABLE'.
syntax error line 1 at position 31 unexpected '.'.


SnowparkSQLException: (1304): 01ad0254-0001-2715-0000-7181f9b37e5e: 001003 (42000): SQL compilation error:
syntax error line 1 at position 12 unexpected 'TABLE'.
syntax error line 1 at position 31 unexpected '.'.

In [ ]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark.functions import *
#desc, abs, count, col, concat, concat_ws, udf, array_construct, countDistinct, when
from snowflake.snowpark.window import Window
from snowflake.snowpark.types import *

import numpy as np
np.float = float

import pandas as pd
from datetime import datetime, timedelta

PREDICT_YEAR = 2023
today = datetime.today()
current_month_start = datetime(today.year, today.month, 1)
previous_month_start = datetime(today.year, today.month-1, 1)


SCHEME = "https"
ACCOUNT = "wa15423.us-east-2.aws"
HOST = "wa15423.us-east-2.aws.snowflakecomputing.com"
PORT = "443"
WAREHOUSE = "ANALYTICS_S"
ROLE = "SVC_DATALAB_ROLE"
DATABASE = "DATA_LAB_TEST"
SCHEMA = "PREDICTOR"

connection_parameters = {
        "account": ACCOUNT,
        "user": 'svc_datalab',
        "password": 'p4Xcwd3JmZz;6j!gs.vtg',
        "role": ROLE,
        "warehouse": WAREHOUSE,
        "database": DATABASE,
        "schema": SCHEMA
      }

session = Session.builder.configs(connection_parameters).create()

query1 = """DELETE FROM DATA_LAB_TEST.PREDICTOR.TR_ORDERS_DEVICE WHERE completedate >= '{0}'"""

query2 = '''INSERT into DATA_LAB_TEST.PREDICTOR.TR_ORDERS_DEVICE 
select 
    A.*, 
    B.FIRST_PURCHASE_YEAR AS FIRST_PURCHASE_YEAR, 
    YEAR(A.completedate)-B.FIRST_PURCHASE_YEAR AS TIME_GAP,
    ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS idx,
    null as SERIALNUMBER_FUZZY,
    ID.PARTYID as PARTYID,
    ID2.PARTYID as SHOPPARTYID
FROM (
    select DISTINCT
    o.client_key AS BPN,
    CASE
        WHEN c.custtypeid = 4 and c.positiontypeid = 157 and c.shopref is not null THEN c.shopref
        ELSE o.custidclientguid
    END GUID,  
    i.item item_sku,
    UPPER(replace(replace(od.serialnumber, ' ', ''), 'USED', '')) as serialnumber_raw,
    CASE 
        WHEN serialnumber_raw LIKE '%-%' AND length(split_part(serialnumber_raw, '-', 1)) NOT IN (3,6,7) 
            
            THEN
            (CASE WHEN (length(split_part(serialnumber_raw, '-', 1)) > length(split_part(serialnumber_raw, '-', 2)))
            THEN UPPER(split_part(serialnumber_raw, '-', 1)) 
            ELSE UPPER(split_part(serialnumber_raw, '-', 2)) END) 
        WHEN serialnumber_raw LIKE '%-%' AND length(split_part(serialnumber_raw, '-', 1)) IN (3,6,7) 
            
            THEN UPPER(LEFT(REGEXP_REPLACE(serialnumber_raw, '([-_+=\.` /;*~])'), 12))
        WHEN serialnumber_raw NOT LIKE '%-%' 
            AND length(serialnumber_raw) BETWEEN 13 AND 14 
            
            THEN UPPER(LEFT(REGEXP_REPLACE(serialnumber_raw, '([-_+=\.` /;*~])'), 12))
        ELSE UPPER(serialnumber_raw)
    END serialnumber_cut, 
    CASE 
        WHEN length(serialnumber_cut) >= 20  
            AND UPPER(LEFT(REGEXP_REPLACE(try_hex_decode_string(serialnumber_cut), '([-\. ])'), 12)) REGEXP '[A-Z0-9]*'
        THEN UPPER(LEFT(REGEXP_REPLACE(try_hex_decode_string(serialnumber_cut), '([-\. ])'), 12))
        WHEN length(serialnumber_cut) = 12 
        THEN serialnumber_cut
    END serialnumber_adj,
    o.completedate,  
    year(o.completedate) as year,
    month(o.completedate) as month,
    CASE WHEN MONTH(o.completedate) <= 3 THEN 1
                        WHEN MONTH(o.completedate) <= 6 THEN 2
                        WHEN MONTH(o.completedate) <= 9 THEN 3
                        WHEN MONTH(o.completedate) <= 12 THEN 4 END AS PREDICT_QUARTER,
    CASE
    WHEN c.custtypeid = 4 THEN
            CASE
            WHEN trunc((to_number(to_char(o.completedate, 'YYYYMMDD'))-to_number(REPLACE(substr(c.birthdate, 0, 10), '-', '')))/ 10000) > 100 THEN 0
            WHEN trunc((to_number(to_char(o.completedate, 'YYYYMMDD'))-to_number(REPLACE(substr(c.birthdate, 0, 10), '-', '')))/ 10000) >= 18 THEN
                     trunc((to_number(to_char(o.completedate, 'YYYYMMDD'))-to_number(REPLACE(substr(c.birthdate, 0, 10), '-', '')))/ 10000)
            ELSE 0
        END
    ELSE -1
    END age,
    c.callday,
    CASE
        WHEN st.type IS NOT NULL THEN st.type
        ELSE 'none'
    END shop_type,
    CASE c.custtypeid
        WHEN 3 THEN o.custidclientguid
        WHEN 4 THEN c.shopref
    END shopguid,
    CASE
            WHEN c.positiontypeid = 157 THEN 1
            WHEN c.custtypeid = 3 THEN 1
            ELSE 0
    END shopowner,
    DECODE(olt.linetypeid, 1, 'SALE', 6, 'TRADEIN') LINETYPE,
    od.listprice sale_list_price,
    od.miscallowance discount_amount,
    substr(i.line, 0, 2) SHORT,
    substr(i.line, 3, 2) MAJOR,
    substr(i.line, 5, 2) MINOR,
    i.curr_item_price item_price,
    d.BASE_MODEL,
    d.tier,
    case when f.company='100' then 'Canada'
        when f.company='130' then 'U.S.'
        when f.company='387' then 'U.K.'
        end as COUNTRY
FROM
    chrome.chromebk.orders o,
    chrome.chromebk.orderdetail od,
    chrome.chromebk.orderlinetype olt,
    data_lab_test.predictor.customer c,
    chrome.chromebk.customer s,
    chrome.chromebk.shoptype st,
    chrome.chromebk.address a,
    chrome.chromesys.recommender_items i,
    data_lab_test.predictor.DI_ITEMS d,
    chrome.chromebk.franchisee_map f
WHERE
    o.client_key = od.client_key
    AND o.clientguid = od.orderidclientguid
    AND od.client_key = olt.client_key
    AND od.clientguid = olt.orderdetailid
    AND o.client_key = c.client_key
    AND o.custidclientguid = c.clientguid
    AND c.client_key = s.client_key
    AND c.shopref = s.clientguid
    AND s.shoptypeid = st.shoptypeid
    AND s.groupid = st.groupid
    AND od.partnumber = i.item
    AND i.item = d.item (+)
    AND c.client_key = a.client_key
    AND c.addrid = a.clientguid
    AND o.client_key = f.client_key (+)
    AND od.isdeleted = 0
    AND olt.isdeleted = 0
    AND olt.linetypeid in (1,6) 
    AND (i.item != 'EEMS343H' 
        OR i.curr_item_price != 1642) 
    AND o.orderstatus = 10
    AND o.ordertypeid = 4
    AND od.quantity > 0
    AND c.isdeleted = 0
    AND s.isdeleted = 0
    AND a.isdeleted = 0
    AND (UPPER(A.NAME1) NOT LIKE '%CASH%'
        OR UPPER(A.NAME2) NOT LIKE '%CASH%'
        OR UPPER(A.NAME1) NOT LIKE 'WALK%'
        OR UPPER(A.NAME2) NOT LIKE 'WALK%')  
    AND o.completedate >= '{0}' 
    AND o.isdeleted = 0 ) A
LEFT JOIN 
    (SELECT USER_ID, MIN(YEAR(TO_TIMESTAMP_NTZ(TIMESTAMP/1000))) AS first_purchase_year 
    FROM DATA_LAB_TEST.RECOMMENDER.RECOMMENDER_EXPORT_VIEW2_ALLTIME 
    GROUP BY USER_ID) B 
    ON (A.BPN || '-' || A.GUID) = B.USER_ID 
LEFT JOIN 
    DATA_LAB_TEST.PREDICTOR.UNIVERSAL_ID_VIEW ID
    ON A.GUID = ID.CLIENTGUID AND A.BPN = ID.CLIENTKEY
LEFT JOIN 
    DATA_LAB_TEST.PREDICTOR.UNIVERSAL_ID_VIEW ID2
    ON A.SHOPGUID = ID2.CLIENTGUID AND A.BPN = ID2.CLIENTKEY'''

table = session.sql(query1.format(previous_month_start))
table.collect()

table = session.sql(query2.format(previous_month_start))
table.collect()

In [2]:
SCHEME = "https"
ACCOUNT = "wa15423.us-east-2.aws"
HOST = "wa15423.us-east-2.aws.snowflakecomputing.com"
PORT = "443"
WAREHOUSE = "ANALYTICS_S"
ROLE = "SVC_DATALAB_ROLE"
DATABASE = "DATA_LAB_TEST"
SCHEMA = "PREDICTOR"

connection_parameters = {
        "account": ACCOUNT,
        "user": 'svc_datalab',
        "password": 'p4Xcwd3JmZz;6j!gs.vtg',
        "role": ROLE,
        "warehouse": WAREHOUSE,
        "database": DATABASE,
        "schema": SCHEMA
      }

session = Session.builder.configs(connection_parameters).create()

In [3]:
all_orders = session.sql("SELECT * FROM TR_ORDERS_DEVICE")

In [4]:
orders_DI = all_orders.filter(col("BASE_MODEL").isNotNull())
DI_list = orders_DI.select(col("PARTYID"), col("SHOPPARTYID")).distinct()
orders_DI = orders_DI.join(orders_DI.groupBy('PARTYID').agg(min(col("COMPLETEDATE")).alias("FIRST_DI_PURCHASE")), ['PARTYID'], 'left')


In [5]:
w_asc = Window.partitionBy('PARTYID').orderBy(col('COMPLETEDATE').asc())

df_DI_upgrades = orders_DI.withColumn("RN", row_number().over(w_asc)).filter(col("rn") > 1).filter(col("YEAR") >= 2014)

df_DI_upgrades = df_DI_upgrades.select(col('BPN'), col('GUID'), col('PARTYID'), col('SHOPPARTYID'), col('SHOPGUID'), col('YEAR'), col('RN'))\
                .withColumn('DATE_PREDICT', date_from_parts(col('YEAR'), 1, 1))\
                .withColumn('PREDICT_QUARTER', ((month(col('DATE_PREDICT'))+2)/3).cast(LongType()))\
                .withColumn("YEAR_PREDICT", col('YEAR'))\
                .withColumn('RN', col('RN'))\
                .withColumn('LABEL', lit(1))\
                .drop('YEAR')
#df_DI_upgrades = df_DI_upgrades.withColumn("PREDICT_QUARTER", df_DI_upgrades["PREDICT_QUARTER"].cast(LongType()))


In [6]:
orders_DI_cust = all_orders.join(orders_DI.select(col("PARTYID"), col('FIRST_DI_PURCHASE')).distinct(), ['PARTYID'], 'inner').filter(col('COMPLETEDATE') > col('FIRST_DI_PURCHASE') )

for year in range(2014, today.year): 
    orders_ngt_cur = orders_DI_cust.filter((col('YEAR') == year))\
                        .groupBy('PARTYID', 'YEAR')\
                        .agg(sum(col("BASE_MODEL").isNotNull().cast('integer')).alias("CNT_DI"))\
                        .filter(col('CNT_DI') == 0).drop('CNT_DI')\
                        .withColumn('DATE_PREDICT', date_from_parts(year, 1, 1))\
                        .withColumn('PREDICT_QUARTER', ((month(col('DATE_PREDICT'))+2)/3).cast(LongType()))\
                        .withColumn('YEAR_PREDICT', lit(year))\
                        .drop('YEAR')
    
    #orders_ngt_cur = orders_ngt_cur.withColumn("PREDICT_QUARTER", orders_ngt_cur["PREDICT_QUARTER"].cast(LongType()))
    
    if year == 2014:
        orders_ngt = orders_ngt_cur
    else:
        '''all_cust = orders_ngt_c.select( col('BPN'), col("GUID"), col('SHOPGUID'))\
                                .union(df_DI_upgrades.filter(col('YEAR_PREDICT')<year)\
                                                    .select( col('BPN'), col("GUID"), col('SHOPGUID'))).distinct()'''
        prev_yr_cust = orders_ngt_c.select( col('PARTYID'), ).distinct()\
                .join(orders_ngt_cur.select( col('PARTYID')).distinct(), ['PARTYID'], 'leftanti')\
                .withColumn('DATE_PREDICT', date_from_parts(year, 1, 1))\
                .withColumn('PREDICT_QUARTER', lit(1))\
                .withColumn('YEAR_PREDICT', lit(year))
        #print(prev_yr_cust.count())
        orders_ngt_cur = orders_ngt_cur.union(prev_yr_cust)
        
        orders_ngt = orders_ngt.union(orders_ngt_cur)
    orders_ngt_c = orders_ngt.cache_result()
    #print(orders_ngt.columns)

w_ngt = Window.partitionBy('PARTYID').orderBy(col('DATE_PREDICT').asc())
orders_ngt = orders_ngt_c.withColumn("RN", row_number().over(w_ngt)).withColumn('LABEL', lit(0))


In [7]:
df_DI_upgrades.groupBy(['YEAR_PREDICT', 'LABEL']).count().orderBy('count').show()

--------------------------------------
|"YEAR_PREDICT"  |"LABEL"  |"COUNT"  |
--------------------------------------
|2014            |1        |3374     |
|2015            |1        |4737     |
|2016            |1        |6447     |
|2017            |1        |8394     |
|2023            |1        |9032     |
|2019            |1        |9567     |
|2018            |1        |10019    |
|2020            |1        |10198    |
|2021            |1        |12690    |
|2022            |1        |13744    |
--------------------------------------



In [8]:
orders_ngt.groupBy(['YEAR_PREDICT', 'LABEL']).count().orderBy('count').show()

--------------------------------------
|"YEAR_PREDICT"  |"LABEL"  |"COUNT"  |
--------------------------------------
|2014            |0        |28506    |
|2015            |0        |48617    |
|2016            |0        |70675    |
|2017            |0        |93440    |
|2018            |0        |115375   |
|2019            |0        |138391   |
|2020            |0        |162652   |
|2021            |0        |189614   |
|2022            |0        |218329   |
--------------------------------------



In [9]:
orders_ngt = orders_ngt.withColumn('BPN', lit(None)).withColumn('GUID', lit(None)).withColumn('SHOPPARTYID', lit(None))\
                        .withColumn('SHOPGUID', lit(None)).select(col('BPN'), col('GUID'), col('PARTYID'), 
                                                                  col('SHOPPARTYID'), col('SHOPGUID'), 
                                                                  col('DATE_PREDICT'), col('PREDICT_QUARTER'), 
                                                                  col('YEAR_PREDICT'), col('RN'), col('LABEL'))
                                                            

In [10]:
df_DI_upgrades.columns, orders_ngt.columns

(['BPN',
  'GUID',
  'PARTYID',
  'SHOPPARTYID',
  'SHOPGUID',
  'DATE_PREDICT',
  'PREDICT_QUARTER',
  'YEAR_PREDICT',
  'RN',
  'LABEL'],
 ['BPN',
  'GUID',
  'PARTYID',
  'SHOPPARTYID',
  'SHOPGUID',
  'DATE_PREDICT',
  'PREDICT_QUARTER',
  'YEAR_PREDICT',
  'RN',
  'LABEL'])

In [11]:
df_train = df_DI_upgrades.union(orders_ngt)

df_train.groupBy('LABEL').count().orderBy('count').show()

---------------------
|"LABEL"  |"COUNT"  |
---------------------
|1        |88202    |
|0        |1065599  |
---------------------



In [12]:
orders_to_predict = all_orders.withColumnRenamed("BPN", 'BPN_all').withColumnRenamed("GUID", 'GUID_all')\
                    .withColumnRenamed("SHOPPARTYID", 'SHOPPARTYID_all').withColumnRenamed("SHOPGUID", 'SHOPGUID_all')\
                    .drop(['PREDICT_QUARTER']).join(df_train, ['PARTYID'], 'right')
orders_to_predict = orders_to_predict.withColumn("GAP_DAY", datediff("day", to_date(orders_to_predict.COMPLETEDATE), to_date(orders_to_predict.DATE_PREDICT)))\
                                    .filter(col('GAP_DAY') >= 0)


In [13]:
orders_to_predict = orders_to_predict.withColumn('BPN', when(col('LABEL')==0, col('BPN_all')).otherwise(col('BPN')))
orders_to_predict = orders_to_predict.withColumn('GUID', when(col('LABEL')==0, col('GUID_all')).otherwise(col('GUID')))
orders_to_predict = orders_to_predict.withColumn('SHOPGUID', when(col('LABEL')==0, col('SHOPGUID_all')).otherwise(col('SHOPGUID')))
orders_to_predict = orders_to_predict.withColumn('SHOPPARTYID', when(col('LABEL')==0, col('SHOPPARTYID_all')).otherwise(col('SHOPPARTYID')))


In [14]:
orders_to_predict.filter(col('SHOPGUID').isNull()).groupBy('LABEL').count().orderBy('count').count()

0

In [15]:
w = Window.partitionBy('PARTYID', 'RN', 'LABEL').orderBy(col('COMPLETEDATE').desc())

# Get information about the previous DI purchase

df_last = orders_to_predict.filter(col("BASE_MODEL").isNotNull()).filter(col('GAP_DAY') >= 1)\
            .withColumn("RN_",row_number().over(w)).filter(col("rn_") == 1).drop("rn_")

df_last = df_last.select(col('BPN'), col('GUID'), col('PARTYID'), col('SHOPPARTYID'), col('SHOPGUID'), 
                         col('DATE_PREDICT'), col('YEAR_PREDICT'), col('RN'), col('LABEL'), col('ITEM_SKU'), 
                         col('ITEM_PRICE'), col('BASE_MODEL'), col('TIER'), col('GAP_DAY'), col('COMPLETEDATE'))

In [16]:
df_last.count()

969045

In [17]:
orders_to_predict.filter(col("BASE_MODEL").isNotNull()).filter(col('GAP_DAY') >= 1).count()

1223677

In [18]:
df_last.groupBy('LABEL').count().orderBy('count').show()

---------------------
|"LABEL"  |"COUNT"  |
---------------------
|1        |65552    |
|0        |903493   |
---------------------



In [19]:
df_DI = orders_to_predict.filter(col("BASE_MODEL").isNotNull()).filter(col('GAP_DAY') >= 1)
df_DI = df_DI.groupBy('PARTYID', 'RN', 'LABEL').count().withColumnRenamed("COUNT", "DI_PURCHASED")\
        .filter(col('DI_PURCHASED') > 0)
df_last = df_last.join(df_DI, ['PARTYID', 'RN', 'LABEL'], 'inner')


In [20]:
w2 = Window.partitionBy(['PARTYID', 'YEAR_PREDICT']).orderBy(col("RN"))
df_last = df_last.withColumn("row",row_number().over(w2)) \
  .filter(col("row") == 1).drop("row")

In [21]:
df_loyal = session.table('TR_SHOP_LOYALTY')
df_last = df_last.join(df_loyal.select(col('BPN'), col('SHOPGUID'), col('LOYALTY')), ['BPN', 'SHOPGUID'], 'left')

In [22]:
df_last.filter(col('LOYALTY').isNull()).count()

895206

In [23]:
df_last = df_last.na.fill({"LOYALTY": 0})

In [24]:
df_last = df_last.join(all_orders.select(col('BPN'), col('COUNTRY')).drop_duplicates(), ['BPN'], 'left')

In [25]:
df_last = df_last.withColumn('SERIALNUMBER_FUZZY', lit(None))
df_last = df_last.withColumn('LAST_UPDATE_DISTANCE', lit(None))
df_last = df_last.withColumn('UPDATE_COUNT', lit(None))
df_last = df_last.withColumn('UPDATE_FREQUENCY', lit(None))
df_last = df_last.withColumn('HAS_SERIAL', lit(None))

In [26]:
df_last = df_last.withColumn('IDX', row_number().over(Window.orderBy(lit(1))))

In [27]:
df_last.write.mode("overwrite").save_as_table("TR_FT_TRAIN")

In [ ]:
df_last.drop(['SERIALNUMBER_FUZZY', 'LAST_UPDATE_DISTANCE', 'UPDATE_COUNT', 'UPDATE_FREQUENCY', 
              'HAS_SERIAL', 'LOYALTY', 'COUNTRY'])

In [32]:
df_train = df_train.drop(['BPN', 'GUID', 'SHOPPARTYID', 'SHOPGUID'])\
                    .join(df_last.select(col('BPN'), col('GUID'), col('SHOPPARTYID'), col('SHOPGUID'), col('PARTYID'),
                                         col('RN'), col('LABEL')), ['PARTYID', 'RN', 'LABEL'], 'inner')
df_train.groupBy('LABEL').count().orderBy('count').show()

---------------------
|"LABEL"  |"COUNT"  |
---------------------
|1        |53206    |
|0        |861288   |
---------------------



In [33]:
df_train.columns

['PARTYID',
 'RN',
 'LABEL',
 'DATE_PREDICT',
 'PREDICT_QUARTER',
 'YEAR_PREDICT',
 'BPN',
 'GUID',
 'SHOPPARTYID',
 'SHOPGUID']

In [34]:
orders_2years = orders_to_predict.filter(col('GAP_DAY') < 4 * 90).filter(col('GAP_DAY') >= 1)
orders_2years = orders_2years.withColumn('GAP_DAY', floor(col('GAP_DAY') / 90))\
                .withColumnRenamed("GAP_DAY", 'GAP')\
                .filter(col("BASE_MODEL").isNull())\
                .filter(col('SHORT').isin(['TS', 'PT', 'HT', 'ES', 'DI']))\
                .groupBy('PARTYID', 'RN', 'LABEL', 'SHORT', 'MAJOR', 'GAP')\
                .agg(sum(col('ITEM_PRICE')).alias("PRICE"))
orders_2years = orders_2years.select("*", concat(col('SHORT'), lit('_'), col('MAJOR'), lit('_'), col('GAP')).alias('PRODUCT_LINE'))\
            .drop(["SHORT", "MAJOR", "GAP"])


In [35]:
product_lines = []
for i in orders_2years.select(col("PRODUCT_LINE")).distinct().collect():
    product_lines.append(i[0])
    
col_list = ["PARTYID", "RN", 'LABEL']
col_list.extend(product_lines)

#orders_2years = orders_2years.drop('SHORT_YEAR')

df_line = orders_2years.pivot("PRODUCT_LINE", product_lines).sum("PRICE")
col_map = dict(zip(df_line.columns, col_list))
df_line = df_line.select([col(c).alias(col_map[c]) for c in df_line.columns])
df_line = df_line.na.fill(value=0, subset=product_lines)

col_list = ["PARTYID", "RN", 'LABEL']
col_list.extend(sorted(product_lines))
df_line = df_line.select(col_list)

In [36]:
df_line.groupBy('LABEL').count().orderBy('count').show()

---------------------
|"LABEL"  |"COUNT"  |
---------------------
|1        |70316    |
|0        |824270   |
---------------------



In [37]:
df_line.write.mode("overwrite").save_as_table("TR_FT_PRODUCT_LINE")

In [39]:
orders_to_predict = all_orders.drop(['GUID', 'PREDICT_QUARTER', 'SHOPPARTYID']).withColumnRenamed("PARTYID", 'PARTYID_IN_SHOP')\
                              .join(df_train, ['BPN', 'SHOPGUID'], 'right')\
                              .withColumn("GAP_DAY", datediff("day", to_date(orders_to_predict.COMPLETEDATE), to_date(orders_to_predict.DATE_PREDICT)))


In [41]:
df_mechs = orders_to_predict.filter(col('GAP_DAY') < 4 * 90).filter(col('GAP_DAY') >= 1)
df_mechs = df_mechs.withColumn('GAP_DAY', floor(col('GAP_DAY') / 90))\
                        .withColumnRenamed("GAP_DAY", 'GAP')\
                        .select(col('PARTYID'), col('BPN'), col('SHOPGUID'), col('PARTYID_IN_SHOP'),\
                                col('RN'), col('LABEL'), col('GAP')).distinct()

df_mechs = df_mechs.groupBy('PARTYID', 'BPN', 'SHOPGUID', 'RN', 'LABEL', 'GAP')\
                    .count().withColumnRenamed("COUNT", "TECH")
df_mechs = df_mechs.pivot('GAP', [0,1,2,3]).sum("TECH")


In [42]:
col_list = ['PARTYID', 'BPN', 'SHOPGUID', 'RN', 'LABEL', 'TECH_0', 'TECH_1', 'TECH_2', 'TECH_3']

col_map = dict(zip(df_mechs.columns, col_list))
df_mechs = df_mechs.select([col(c).alias(col_map[c]) for c in df_mechs.columns])
df_mechs = df_mechs.na.fill(value=0, subset=['TECH_0', 'TECH_1', 'TECH_2', 'TECH_3'])


In [43]:
df_mechs.groupBy('LABEL').count().orderBy('count').show()

---------------------
|"LABEL"  |"COUNT"  |
---------------------
|1        |51985    |
|0        |792216   |
---------------------



In [44]:
df_mechs.write.mode("overwrite").save_as_table("TR_FT_MECHANICS")